### 目標: 爬取mlb受傷球員資料
[點我spotrac-爬取此網站](https://www.spotrac.com/mlb/disabled-list/2017/)
* requests與網站取得連線
* 

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [30]:
url="http://www.spotrac.com/mlb/disabled-list/2018/"

html=requests.get(url)
sp=BeautifulSoup(html.text,"html.parser")

#因為此網站的隊名在option的標籤裡,把它存入字典
table=sp.select("select option")
teams_name={}
for i in range(13,43):
    #取得隊伍完整名稱與簡寫
    team_logogram=table[i].text
    team_name=table[i].get("value")
    teams_name[team_name]=team_logogram

In [31]:
teams_name

{'arizona-diamondbacks': 'ARI',
 'atlanta-braves': 'ATL',
 'baltimore-orioles': 'BAL',
 'boston-red-sox': 'BOS',
 'chicago-cubs': 'CHC',
 'chicago-white-sox': 'CHW',
 'cincinnati-reds': 'CIN',
 'cleveland-indians': 'CLE',
 'colorado-rockies': 'COL',
 'detroit-tigers': 'DET',
 'houston-astros': 'HOU',
 'kansas-city-royals': 'KC',
 'los-angeles-angels-of-anaheim': 'LAA',
 'los-angeles-dodgers': 'LAD',
 'miami-marlins': 'MIA',
 'milwaukee-brewers': 'MIL',
 'minnesota-twins': 'MIN',
 'new-york-mets': 'NYM',
 'new-york-yankees': 'NYY',
 'oakland-athletics': 'OAK',
 'philadelphia-phillies': 'PHI',
 'pittsburgh-pirates': 'PIT',
 'san-diego-padres': 'SD',
 'san-francisco-giants': 'SF',
 'seattle-mariners': 'SEA',
 'st.-louis-cardinals': 'STL',
 'tampa-bay-rays': 'TB',
 'texas-rangers': 'TEX',
 'toronto-blue-jays': 'TOR',
 'washington-nationals': 'WSH'}

In [91]:
for j in range(2015,2018):
    #建立年份資料夾
    os.mkdir("C:/Users/wen/Desktop/python/Web-Crawler/"+str(j))
    #將每年每隊資料都獲得
    for i in teams_name.keys():
        data=pd.read_html("https://www.spotrac.com/mlb/disabled-list/"+str(j)+"/"+i)[3]
        
        #刪除最後一行
        data.drop(index=(len(data)-1),inplace=True)
        data.dropna(axis=1,inplace=True)
        data.drop(columns=["Cash EarnedWhile on DL","Days"],inplace=True)

        #留下隊員的名字
        data2.iloc[:,0]=data2.iloc[:,0].apply(lambda x:' '.join(x.split(',')[0].split(' ')[:-1]))

        #將日期轉換成兩個欄位開始、結束
        data2=data.rename(columns={"Status":"Start"})
        data2["Start"]=data2["Start"].apply(lambda x:x.split('|')[-1])
        data3=data2.copy()
        data2["Start"]=data2["Start"].apply(lambda x:x.split('|')[-1].split("-")[0])
        data2["End"]=data3["Start"].apply(lambda x:x.split('|')[-1].split("-")[-1])
        #受傷沒有終止日給定球季結束日(11/1)
        data2["End"]=data2["End"].apply(lambda x:"11/1" if x=="" else x)

        #日期格式轉換
        data2.Start=data2.Start.apply(lambda x:str(j)+'-'+'{:02d}'.format(int(x.split('/')[0]))+'-'+'{:02d}'.format(int(x.split('/')[1])))
        data2.End=data2.End.apply(lambda x:str(j)+'-'+'{:02d}'.format(int(x.split('/')[0]))+'-'+'{:02d}'.format(int(x.split('/')[1])))

        #照受傷期間排列
        data2_sort=data2.sort_values('Start')
        #因為每個在每季都會不同,要換成該季的
        data2_sort.Team=data2_sort.Team.apply(lambda x:teams_name.get(i))
        
        #照年份隊名簡寫存檔
        data2_sort.to_csv("C:/Users/wen/Desktop/python/Web-Crawler/"+str(j)+"/"+teams_name.get(i)+".csv",index=False,encoding='utf-8')